# This notebook tries to improve an existing classification model using yolo to crop the image before predicting the class


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Jeremynadal33/classify_stanford_dogs/blob/master/optimize_classification_with_yolo.ipynb)

This notebook presents :
* Processing of yolo outputs
* Downloading and study of the average size of a dog in the Stanford dataset
* Make a function that manage a photo to optimize the predicton